# 加载新闻数据的相似度

## 准备工作

In [3]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

## 加载数据

In [36]:
%%time

# 加载 CSV 文件
df = pd.read_csv('./test-100.csv')
# 显示前10条
df.head(10)

CPU times: user 3.53 ms, sys: 0 ns, total: 3.53 ms
Wall time: 3.07 ms


,text1,text2
0,本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1...,可穿戴技术十大设计原则
1,2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而...,经济学人：智能手机将成为“真正的个人电脑”
2,雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打...,雅虎宣布剥离阿里巴巴股份
3,2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线...,51信用卡管家，预计2015年放贷额度远超30亿
4,目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言...,如何选择正确的编程语言进行学习
5,受众在哪里，媒体就应该在哪里，媒体的体制、内容、技术就应该向哪里转变。媒体融合关键是以人为本...,媒体融合关键是以人为本
6,中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希...,自行车不能带上火车
7,今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实...,北京警方确认李代沫吸毒被捕(图)
8,党的十八大以来，以习近平同志为总书记的党中央高瞻远瞩战略谋划，着力创新发展理念，大力建设生态...,生态兴则文明兴生态衰则文明衰
9,昨晚，中联航空成都飞北京一架航班被发现有多人吸烟。后因天气原因，飞机备降太原机场。几名乘客在...,成都飞北京航班多人吸烟机组人员与未吸烟乘客冲突


## 文档编码

In [7]:
%%time

from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("/models/bge-large-zh-v1.5")

CPU times: user 1.19 s, sys: 626 ms, total: 1.82 s
Wall time: 1.52 s


In [8]:
text=df['text1']
text

0     本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1...
1     2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而...
2     雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打...
3     2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线...
4     目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言...
                            ...                        
94    银行股连创新高，踩了王亚伟，捧了史玉柱。王亚伟“不爱”银行股，甚至传“郭树清因此斥责王亚伟”...
95    1月20日上午，湖南湘乡市一老式爆米花机突然爆炸，碎片击中一7岁小孩头部，致其当场身亡。男孩...
96    5月9日，利得财富管理集团正式对外宣布，旗下基于利得财富官方微信(微信号：leadcf)端口...
97    “有些人并没看清其中深意。要么认为反腐是杀鸡儆猴，抓几个贪官为了聚拢民心；要么就认为这是高层...
98    欠债还钱，自古天理。为什么有那么一些业主，自己可以大手大脚随便挥霍，就是不肯把仨瓜俩枣的工钱...
Name: text1, Length: 99, dtype: object

In [11]:
%%time

encoded_data = encoder.encode(text)

CPU times: user 4.3 s, sys: 130 ms, total: 4.43 s
Wall time: 4.44 s


## 创建索引

In [12]:
%%time

index = faiss.IndexIDMap(faiss.IndexFlatIP(1024))
index.add_with_ids(encoded_data, np.arange(len(text)))

CPU times: user 0 ns, sys: 404 µs, total: 404 µs
Wall time: 443 µs


## 搜索

In [15]:
def search(query, k=1):
    query_vector = encoder.encode([query])
    top_k = index.search(query_vector, k)
    print(top_k)
    return [
        text[_id] for _id in top_k[1][0]
    ]

In [17]:
%%time

search('李代沫',k=1)

(array([[0.51555777]], dtype=float32), array([[7]]))
CPU times: user 20.6 ms, sys: 17.3 ms, total: 37.8 ms
Wall time: 39.9 ms


['今天有传在北京某小区，一光头明星因吸毒被捕的消息。下午北京警方官方微博发布声明通报情况，证实该明星为李代沫。李代沫伙同另外6人，于17日晚在北京朝阳区三里屯某小区的暂住地内吸食毒品，6人全部被警方抓获，且当事人对犯案实施供认不讳。']

In [21]:
%%time

search('交通',k=5)

(array([[0.4607549 , 0.45903063, 0.44265407, 0.4400144 , 0.41214722]],
      dtype=float32), array([[ 6, 30, 54, 47, 46]]))
CPU times: user 22.8 ms, sys: 16.4 ms, total: 39.2 ms
Wall time: 38.8 ms


['中国铁路总公司消息，自2015年1月5日起，自行车不能进站乘车了。骑友大呼难以接受。这部分希望下车就能踏上骑游旅程的旅客，只能先办理托运业务，可咨询12306客服电话，就近提前办理。运费每公斤价格根据运输里程不同而不同。',
 '文章称，30年收费政策功不可没，使公路发展摆脱了资金不足的束缚。世界各国政府在提供公路基础设施过程中都面临这样一个艰难的选择，就是让全民通过付税还是让使用者通过付费来承担这项长期的责任。',
 '春节临近，内地客赴港“扫货游”掀热潮，本周末已迎来客流高峰，预计下周末再创新高。岁末年初，以购物为主要内容的“扫货游”成为出境游的重要内容，其中港澳依旧是内地客最热门出游目的地之一。如直达香港的广九直通车，较平时激增逾6成，超1.2万人次。',
 '近年来，逢雨必涝、逢涝必瘫，几成我国城市通病。上周，中国青年报对全国31个省（区、市）5375人进行的调查显示，91.6%的人关注所在城市的排水问题；84.7%的受访者赞同，城市现代化更表现在地面之下，应加大地下民生工程建设投入。',
 '8月1日，全国性地方债审计全面开闸。审计的背后，是部分地方政府盲目举债的隐忧及无力还债的现实。媒体披露审计的15个目标省会城市中，债务压力排名前10为：南京、成都、广州、合肥、昆明、长沙、武汉、哈尔滨、西安和兰州。']

In [22]:
%%time

search('科技',k=3)

(array([[0.4321493 , 0.40868598, 0.39781177]], dtype=float32), array([[11, 82, 28]]))
CPU times: user 19.5 ms, sys: 20.1 ms, total: 39.5 ms
Wall time: 39.6 ms


['11日下午，中共中央政治局常委、中央书记处书记刘云山登门看望了国家最高科技奖获得者于敏、张存浩。刘云山指出，广大科技工作者要学习老一辈科学家求真务实的钻研精神，淡泊名利、潜心科研，努力创造更多一流科研成果。',
 '仔细一算，上海的互联网公司不乏成功案例，但最终成为BAT一类巨头的几乎没有，这也能解释为何纳税百强的榜单中鲜少互联网公司的身影。有一类是被并购，比如：易趣、土豆网、PPS、TrinitiInteractive、PPTV、1号店等；有一类是数年偏安于细分市场',
 '李克强总理5月30日主持召开国务院常务会议，决定对国务院已出台政策措施落实情况开展全面督查。这既是中央政府贯彻了科学发展观，也是地方政府树立了正确的政绩观。政策和落实，进行辨证统一之后，改革成果指数才能寻求到最大公约数。']

## 保存索引

In [23]:
%%time

path = './news-100.index'

# Save index
faiss.write_index(index, path)

CPU times: user 0 ns, sys: 671 µs, total: 671 µs
Wall time: 467 µs


## 增加新数据并搜索

### 加载文件

In [49]:
%%time

# 加载索引文件
index = faiss.read_index(path)
# 加载 CSV 文件
df = pd.read_csv('./test-100.csv')

CPU times: user 2.97 ms, sys: 0 ns, total: 2.97 ms
Wall time: 2.32 ms


### 加入新数据

In [50]:
# 要追加的数据
new_data = {
    'text1': ['2024年4月26日，中共中央政治局委员、外交部长王毅在北京同美国国务卿布林肯举行会谈时，就中美关系表示，去年11月，习近平主席和拜登总统在旧金山会晤，共同擘画了“旧金山愿景”。在两国元首的掌舵领航下，中美关系总体止跌企稳，双方各领域的对话增多、合作增多、积极面增多，两国人民和国际社会都对此表示欢迎。但另一方面，中美关系的负面因素仍在上升积聚，面临各种干扰破坏，中国的正当发展权利遭到无理打压，中方的核心利益不断受到挑战。'], 
    'text2': ['王毅谈中方对中美关系的“三个一以贯之”']}
# 创建要追加的 DataFrame
new_df = pd.DataFrame(new_data)
# 追加数据
df = pd.concat([df, new_df], ignore_index=True)

df

,text1,text2
0,本文总结了十个可穿戴产品的设计原则，而这些原则，同样也是笔者认为是这个行业最吸引人的地方：1...,可穿戴技术十大设计原则
1,2007年乔布斯向人们展示iPhone并宣称“它将会改变世界”，还有人认为他在夸大其词，然而...,经济学人：智能手机将成为“真正的个人电脑”
2,雅虎发布2014年第四季度财报，并推出了免税方式剥离其持有的阿里巴巴集团15％股权的计划，打...,雅虎宣布剥离阿里巴巴股份
3,2014年，51信用卡管家跟宜信等P2P公司合作，推出线上信贷产品“瞬时贷”，其是一种纯在线...,51信用卡管家，预计2015年放贷额度远超30亿
4,目前世界上有着几百种编程语言，我应该学哪个?如何选择“正确”的编程语言进行学习?我所学的语言...,如何选择正确的编程语言进行学习
...,...,...
95,1月20日上午，湖南湘乡市一老式爆米花机突然爆炸，碎片击中一7岁小孩头部，致其当场身亡。男孩...,爆米花机爆炸，碎铁砸死7岁男童
96,5月9日，利得财富管理集团正式对外宣布，旗下基于利得财富官方微信(微信号：leadcf)端口...,利得财富移动互联网金融O2O微官网平台正式上线
97,“有些人并没看清其中深意。要么认为反腐是杀鸡儆猴，抓几个贪官为了聚拢民心；要么就认为这是高层...,新设计师习近平•反腐篇
98,欠债还钱，自古天理。为什么有那么一些业主，自己可以大手大脚随便挥霍，就是不肯把仨瓜俩枣的工钱...,党报批欠薪业主：自己挥霍就不肯把工钱给农民工


### 加入索引

In [51]:
%%time

new_data['text1']
new_encoded_data = encoder.encode(new_data['text1'])
index.add_with_ids(new_encoded_data, len(df)-1)

CPU times: user 112 ms, sys: 19.8 ms, total: 132 ms
Wall time: 150 ms


In [53]:
text=df['text1']

### 搜索

In [56]:
%%time

search('中美关系',k=1)


(array([[0.62922513]], dtype=float32), array([[99]]))
CPU times: user 19.2 ms, sys: 20.8 ms, total: 40 ms
Wall time: 39 ms


['2024年4月26日，中共中央政治局委员、外交部长王毅在北京同美国国务卿布林肯举行会谈时，就中美关系表示，去年11月，习近平主席和拜登总统在旧金山会晤，共同擘画了“旧金山愿景”。在两国元首的掌舵领航下，中美关系总体止跌企稳，双方各领域的对话增多、合作增多、积极面增多，两国人民和国际社会都对此表示欢迎。但另一方面，中美关系的负面因素仍在上升积聚，面临各种干扰破坏，中国的正当发展权利遭到无理打压，中方的核心利益不断受到挑战。']